In [1]:
import json
import pandas as pd
import load_dataset as load_dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt


import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

import torch
from torch.utils.data import Dataset, DataLoader

# annotation_coco_path = "/root/dataset/anime_face_landmark_20230912/annotations/person_keypoints_Train.json"
# data_path = "/root/dataset/anime_face_landmark_20230912/images"
# dataset_list = load_dataset.cocokeypoints_list_converter(annotation_coco_path)
# training_samples, valid_samples = load_dataset.train_test_split(dataset_list,0.05)
# train_numpy_dataset = load_dataset.AugmentFaceKeypointDataset(valid_samples,data_path,3)

In [2]:
from model_tester import model_test
import sys
import importlib
import config

sys.path.append('./models')
MODEL_FILE = "resnet18"
module = importlib.import_module(MODEL_FILE)
model = module.LandmarkDetector().to(config.DEVICE)
# 白黒画像で学習したモデル
# model_path = "/root/Cloud/deeplearning_results/mlruns/822573913619159784/83160e827531423ab9170affe543e425/artifacts/models/model_epoch_10000.pth"
# カラー画像で学習したモデル
# model_path = "/root/Cloud/deeplearning_results/mlruns/822573913619159784/33396b9a51f74a36b9e0a40aa109be5d/artifacts/models/model_epoch_10000.pth"
model_path = "/root/Cloud/deeplearning_results/mlruns/822573913619159784/bf161b33e5a74d63a5a674b6a826b8db/artifacts/models/model_epoch_7000.pth"
valid_image_names = [
    "images/valid_radiantLight_gitv1.5.4_0901_132440_1.jpg",
    "images/valid_radiantLight_gitv1.5.4_0901_132508_0.jpg",
    "images/valid_radiantLight_gitv1.5.4_0901_132534_1.jpg",
    "images/valid_radiantLight_gitv1.5.4_0901_132628_1.jpg"
    ]
# model_test(model_test(model,model_path,f"{config.DATASET_PATH}/images",valid_image_names,f"./"))

In [3]:
# def model_test(model,model_path,dataset_path,image_list,save_image_dir):
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
for image_name in valid_image_names:
    with torch.no_grad():
        # image = cv2.imread(f"/root/dataset/anime_face_landmark_20230912/images/images/AnythingV5Ink_gitv1.5.4_0901_140932_0.jpg")
        # image = cv2.imread(f"/root/source/image_convet/mizore.png") # カラー
        # image = cv2.imread(f"/root/source/image_convet/free.png") # カラー
        
        # image = cv2.imread(f"/root/source/image_convet/output_free.png") # 白黒画像
        # image = cv2.imread(f"/root/source/image_convet/mizore_3.png") # 白黒画像
        
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        orig_image = image.copy()
        shape_y,shape_x,_ = orig_image.shape
        image = cv2.resize(image, (config.RESIZE, config.RESIZE))
        print(image.shape)
        image = image / 255.0
        image = np.transpose(image, (2, 0, 1))
        # image = np.transpose(image)
        image = torch.tensor(image, dtype=torch.float)
        image = image.unsqueeze(0).to(config.DEVICE)
        outputs = model(image)
        print(outputs)
        outputs = outputs.cpu().detach().numpy()
        outputs = outputs.reshape(-1, 2)
        # keypoints = keypoints.reshape(-1, 2)
        keypoints = outputs * [shape_x,shape_y]
        # plt.subplot(3, 4, i+1)
        plt.imshow(orig_image, cmap='gray')
        for p in range(keypoints.shape[0]):
            plt.plot(keypoints[p, 0], keypoints[p, 1], 'r.')
            plt.text(keypoints[p, 0], keypoints[p, 1], f"{p}")
        plt.axis('off')
        # image_name = image_name.replace("images/","")
        # plt.savefig(f"{save_image_dir}/valid_{image_name}")
        plt.show()
        plt.close()


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.